In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
import sys
%matplotlib inline

Using Theano backend.


In [2]:
sys.path.append("../src/")

In [3]:
import data

Welcome to use Wind Quant API for Python (WindPy)!
You can use w.menu to help yourself to create commands(WSD,WSS,WST,WSI,WSQ,...)!

COPYRIGHT (C) 2016 WIND HONGHUI INFORMATION & TECHKNOLEWDGE CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [4]:
import time_series_momentum

In [5]:
sns.set(style="darkgrid")

# 预测收益率

In [6]:
contract = "000300.SH"

In [7]:
day_return = data.get_contract_data(contract)

In [8]:
day_return.head()

,000300.SH
1990-12-19 00:00:00.005,NaN
1990-12-20 00:00:00.005,NaN
1990-12-21 00:00:00.005,NaN
1990-12-24 00:00:00.005,NaN
1990-12-25 00:00:00.005,NaN


In [9]:
day_return = day_return.pct_change()

In [10]:
day_return.dropna(inplace=True)

In [11]:
day_return.head()

,000300.SH
2002-01-07 00:00:00.005,-0.010916
2002-01-08 00:00:00.005,-0.007196
2002-01-09 00:00:00.005,-0.015525
2002-01-10 00:00:00.005,0.006770
2002-01-11 00:00:00.005,-0.024544


In [12]:
split_date = datetime(2010, 1, 1)
train_return = day_return[day_return.index < split_date]
test_return = day_return[day_return.index >= split_date]

In [13]:
train_return.tail()

,000300.SH
2009-12-25 00:00:00.005,-0.004081
2009-12-28 00:00:00.005,0.015665
2009-12-29 00:00:00.005,0.006412
2009-12-30 00:00:00.005,0.016602
2009-12-31 00:00:00.005,0.004728


In [14]:
test_return.head()

,000300.SH
2010-01-04 00:00:00.005,-0.011314
2010-01-05 00:00:00.005,0.008149
2010-01-06 00:00:00.005,-0.006260
2010-01-07 00:00:00.005,-0.019841
2010-01-08 00:00:00.005,0.002499


In [46]:
def create_sequence_dataset(dataset, look_back=12):
    dataset = dataset.values
    dataset = dataset.astype('float32')
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [47]:
look_back = 12

In [48]:
trainX, trainY = create_sequence_dataset(train_return, look_back)

In [49]:
trainX.shape, trainY.shape

((1922, 12), (1922,))

In [50]:
testX, testY = create_sequence_dataset(test_return, look_back)

In [51]:
testX.shape, testY.shape

((1701, 12), (1701,))

In [52]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [53]:
trainX.shape, testX.shape

((1922, 1, 12), (1701, 1, 12))

In [61]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_dim=look_back))
model.add(Dense(1))

ValueError: Cannot compute test value: input 0 (lstm_input_6) of Op Elemwise{second,no_inplace}(lstm_input_6, DimShuffle{x,x,x}.0) missing default value

TypeError: __init__() takes at least 2 arguments (1 given)